In [ ]:
%cd ~/Documents/cvd-predictor
import polars as pl
import numpy as np

In [ ]:
# df: pl.DataFrame = pl.read_csv("data/raw/LLCP2023.csv")
# df.write_parquet("data/intermediate/LLCP2023.parquet")
df: pl.DataFrame = pl.read_parquet("data/intermediate/LLCP2023.parquet")

In [ ]:
variables: pl.DataFrame = pl.read_csv("data/raw/selected_variables.csv")
df: pl.DataFrame = df[variables["Variable"]]
df.columns = variables["Renamed"]
df

In [ ]:
df: pl.DataFrame = df.with_columns(
    pl.when(
        (pl.col("BlindOrVisionDifficulty") == 1)
        | (pl.col("DeafOrHardOfHearing") == 1)
    )
    .then(1)
    .when(
        (pl.col("BlindOrVisionDifficulty") == 2)
        | (pl.col("DeafOrHardOfHearing") == 2)
    )
    .then(0)
    .otherwise(None) 
    .cast(pl.Int8)  
    .alias("Sensory Impairments")
)
df: pl.DataFrame = df.drop(["BlindOrVisionDifficulty", "DeafOrHardOfHearing"])

In [ ]:
df: pl.DataFrame = df.with_columns(
    pl.when(
        (pl.col("FluVaxLast12") == 1)
        | (pl.col("PneumoVaxEver") == 1)
    )
    .then(1)
    .when(
        (pl.col("FluVaxLast12") == 2)
        | (pl.col("PneumoVaxEver") == 2)
    )
    .then(0)
    .otherwise(None) 
    .cast(pl.Int8)  
    .alias("Vaccinated")
)
df: pl.DataFrame = df.drop(["FluVaxLast12", "PneumoVaxEver"])

In [ ]:
df: pl.DataFrame = df.with_columns(
    pl.when(
        (pl.col("DifficultyWalking") == 1)
        | (pl.col("DifficultyDressingBathing") == 1)
        | (pl.col("DifficultyErrands") == 1)
    )
    .then(1)
    .when(
        (pl.col("DifficultyWalking") == 2)
        | (pl.col("DifficultyDressingBathing") == 2)
        | (pl.col("DifficultyErrands") == 2)
    )
    .then(0)
    .otherwise(None) 
    .cast(pl.Int8)  
    .alias("Mobility")
)
df: pl.DataFrame = df.drop(["DifficultyWalking", "DifficultyDressingBathing", "DifficultyErrands"])

In [ ]:
df: pl.DataFrame = df.with_columns(
    pl.when(pl.col("SmokerStatus") == 9)
      .then(None)
      .otherwise(pl.col("SmokerStatus"))
      .alias("SmokerStatus")
)

In [ ]:
df: pl.DataFrame = df.with_columns(
    pl.when(pl.col("HaveHighCholesterol")==1)
    .then(1)
    .when(
        (pl.col("HaveHighCholesterol")==2)
    )
    .then(0)
    .otherwise(None)
    .cast(pl.Int8)
    .alias("HaveHighCholesterol")
)

In [ ]:
LAST_CHECKUP: dict[int, str] = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    7: None,
    8: None,
    9: None,
}

In [ ]:
df=df.drop_nulls(["GeneralHealth","PhysicalHealthDays","MentalHealthDays","HadSkinCancer","HadArthritis","HadDiabetes"])
df = df.with_columns(
    pl.col("Sex").map_elements(lambda x: 0 if x == 2 else x, return_dtype=pl.Int8),
    pl.col("GeneralHealth").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("PhysicalHealthDays").map_elements(lambda x: None if x >30 else x, return_dtype=pl.Int8),
    pl.col("MentalHealthDays").map_elements(lambda x: None if x >30 else x, return_dtype=pl.Int8),
    pl.col("LastCheckupTime").map_elements(LAST_CHECKUP.get, return_dtype=pl.Int8),
    pl.col("PhysicalActivities").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadAsthma").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadSkinCancer").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadCOPD").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadDepressiveDisorder").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadKidneyDisease").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadArthritis").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HadDiabetes").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("AgeCategory").map_elements(lambda x: None if x == 14 else x, return_dtype=pl.Int8),
    pl.col("AlcoholDrinkers").map_elements(lambda x: None if x == 7 or x == 9 else x, return_dtype=pl.Int8),
    pl.col("HeightInMeters") / 100,
    pl.col("WeightInKilograms") / 100,
    pl.col("BMI") / 100,
)

In [ ]:
df: pl.DataFrame = df.with_columns(
    pl.when(
        (pl.col("HadHeartAttack") == 1)
        | (pl.col("HadAngina") == 1)
        | (pl.col("HadStroke") == 1)
    )
    .then(1)
    .when(
        (pl.col("HadHeartAttack") == 2)
        | (pl.col("HadAngina") == 2)
        | (pl.col("HadStroke") == 2)
    )
    .then(0)
    .otherwise(None) 
    .cast(pl.Int8)  
    .alias("CVD")
)
df: pl.DataFrame = df.drop(["HadHeartAttack", "HadAngina", "HadStroke"])


In [ ]:
df=df.drop_nulls(["CVD"])
df

In [ ]:
df.write_csv("data/intermediate/heart_cdc_2023.csv")
df